In [ ]:
#imports
import numpy as np
from Bio import Phylo
import pandas as pd

# Background

The purpose of this project is to investigate if codon bias is an indicator for phylogenetic relationships. If codon bias does prove to be an indicator, this could be used to increase understanding of the relationship between different organisms. A csv file containing the codon bias for many different organisms was taken from Kaggle (source?). TO make the project more manageable, rodents were selected to be the focus on the project. The data was analyzed by creating a data frame and using pandas techniques from BIOL300 such as reading in a file and indexing a data frame. To create a phylogenetic tree, the algorithm UPGMA from BIOL301 was used. 

# Part 1 - Preparing the Data

First, the csv file must be read into the file as a data frame. This was done using Pandas a skills from BIOL300. 

In [ ]:
#read file into a data frame
f = open('codon_usage.csv', 'r')
df = pd.read_csv(f);

The csv files is quite large, so it was decreased in size to be more manageable. Rodents were selected as the subject of this study, so all animals labeled as in the rodent kingdom were added to a new data frame. Then, since this data included codon bias for mitochondrial DNA, that was also removed to limit the scope of this project. 

In [ ]:
#parse out only columns labeled as 'rod' (rodents)
rodents_with_mit = df[df['Kingdom'] == 'rod']

#remove the mitocondrial DNA by using the DNAtype column
rodents = rodents_with_mit[rodents_with_mit['DNAtype'] == 0]

In [ ]:
#limit the number of rodents and print out dataframe
rodents = rodents[:5]
rodents

The csv file contained codon bias across the entire genome. It was decided to normalize these percentages for each amino acid. This removed any amino acid bias between species and focuses just on the codon bias. To accomplish this, a dictionary was created to correlate each codon to a specific amino acid (source?). Multiple checks were performed to make sure all amino acids and codons were present and there were no typos. 

In [ ]:
#create dictionary of amino acids and corresponding codons
aas = {'Phe': ['UUU','UUC'], 'Leu': ['UUA','UUG','CUU','CUC','CUA','CUG'], 'Ile': ['AUU','AUC','AUA'], 'Met': ['AUG'], 'Val': ['GUU','GUC','GUA','GUG'],
       'Ser': ['UCU','UCC','UCA','UCG','AGU','AGC'], 'Pro': ['CCU','CCC','CCA','CCG'], 'Thr': ['ACU','ACC','ACA','ACG'], 'Ala': ['GCU','GCC','GCA','GCG'], 
       'Tyr': ['UAU','UAC'], '***': ['UAA','UAG','UGA'], 'Trp': ['UGG'], 'His': ['CAU','CAC'], 'Gln': ['CAA','CAG'], 'Asn': ['AAU','AAC'], 'Lys': ['AAA','AAG'],
       'Cys': ['UGU','UGC'], 'Arg': ['CGU','CGC','CGA','CGG','AGA','AGG'], 'Gly': ['GGU','GGC','GGA','GGG'], 'Asp': ['GAU','GAC'], 'Glu': ['GAA','GAG']}

In [ ]:
#check that all amino acids are in dictionary (20+stop = 21)
len(aas)

In [ ]:
#check that all codons are included in dictionary (should be 64)
count = 0
for i in aas:
    count += len(aas[i])
    
print(count)

Once the dictionary was completed and checked, it was used to standardize by amino acid. By iterating through the dictionary, it was established how often each codon was used to create a certain amino acid for each animal. These new percentages were then added to the data frame, replacing the former percentages which accounted for the whole genome. The data frame was then printed to make sure the data frame was updated

In [ ]:
#standarize by amino acid

#loop through each animal
for animal in rodents['SpeciesName']:
    species_bool = rodents['SpeciesName'] == animal
    species = rodents[species_bool]
    
    #loop through all amino acids
    for aa in aas:
        #initalize total percent of amino acid
        total = 0 
        
        #loop through codons and add up percentages
        for codon in aas[aa]:
            total += species[codon]
        
        #loop through codons, normalize by amino acid, and replace in data frame
        for codon in aas[aa]:
            rodents.loc[species_bool, [codon]] = species[codon]/total

In [ ]:
#print rodents to make sure it updated 
rodents

Once the data was cleaned and standardized by amino acid, a distance function was created to determine the distance between two species. It was decided to use a sum of squares. The squared difference between each codon was added together and the total difference returned to determine how different codon bias between species is.

In [ ]:
def species_diff(species1, species2):
    # finds the difference between the codon bias of two species
    
    # step a counting value to 0
    diff = 0
    
    animal1 = rodents[rodents['SpeciesName'] == species1]
    animal2 = rodents[rodents['SpeciesName'] == species2]
    
    #loop through all amino acids
    for aa in aas:
        #loop through all codons
        for codon in aas[aa]:
            #add the squared difference between the species codon bias squared 
            diff += (animal1[codon].values[0] - animal2[codon].values[0])**2
    
    #return the total difference between two speices
    return diff         

# Part 2 - UPGMA

In [ ]:
class Node:
    def __init__(self):
        self.left = None
        self.right = None
        self.val = None

In [ ]:
def differences(seq1, seq2):
    '''Counts the number of pairwise differences between
    two sequences'''
    
    count = 0
    
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            count += 1
            
    return count

In [ ]:
human =   "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
chimp =   "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
gorilla = ".VLSPADKTNVKAAWGKVGAHAGDYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
cow =     "MVLSAADKGNVKAAWGKVGGHAAEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVNFKLLSHSLLVTLASHLPSDFTPAVHASLDKFLANVSTVLTSKYR"
horse =   "MVLSAADKTNVKAAWSKVGGHAGEYGAEALERMFLGFPTTKTYFPHF.DLSHGSAQVKAHGKKVGDALTLAVGHLDDLPGALSNLSDLHAHKLRVDPVNFKLLSHCLLSTLAVHLPNDFTPAVHASLDKFLSSVSTVLTSKYR"  
donkey =  "MVLSAADKTNVKAAWSKVGGNAGEFGAEALERMFLGFPTTKTYFPHF.DLSHGSAQVKAHGKKVGDALTLAVGHLDDLPGALSNLSDLHAHKLRVDPVNFKLLSHCLLSTLAVHLPNDFTPAVHASLDKFLSTVSTVLTSKYR" 
rabbit =  ".VLSPADKTNIKTAWEKIGSHGGEYGAEAVERMFLGFPTTKTYFPHF.DFTHGSZQIKAHGKKVSEALTKAVGHLDDLPGALSTLSDLHAHKLRVDPVNFKLLSHCLLVTLANHHPSEFTPAVHASLDKFLANVSTVLTSKYR"
carp =    "MSLSDKDKAAVKGLWAKISPKADDIGAEALGRMLTVYPQTKTYFAHWADLSPGSGPVKKHGKVIMGAVGDAVSKIDDLVGGLAALSELHAFKLRVDPANFKILAHNVIVVIGMLYPGDFPPEVHMSVDKFFQNLALALSEKYR"

In [ ]:
animals = [carp, cow, donkey, horse, human, gorilla, rabbit]
nodes = ["carp", "cow", "donkey", "horse", "human", "gorilla", "rabbit"]

In [ ]:
def create_diff_dict(animals, nodes):
    '''Creates a matrix of differences between the given species.
    Stored in a dictionary of dictionary based on the indices of 
    the provided list.'''
    
    diff_dict = {}

    for i, animal1 in enumerate(nodes):
        for j, animal2 in enumerate(nodes):
            if(animal1 == animal2):
                continue
                
            diff = differences(animals[i], animals[j])

            if(not animal1 in diff_dict):
                diff_dict[animal1] = {}

            if(not animal2 in diff_dict):
                diff_dict[animal2] = {}

            diff_dict[animal1][animal2] = diff
            diff_dict[animal2][animal1] = diff
        
    return diff_dict

In [ ]:
create_diff_dict(animals, nodes)

In [ ]:
def pair_group(diff_dict):
    """ given a matrix of differences, returns the indices of the closest two related organisms"""
    
    min_val = np.inf
    index_1 = 0
    index_2 = 0
    
    for i in diff_dict.keys():
        for j in diff_dict[i].keys():
            if diff_dict[i][j] < min_val:
                min_val = diff_dict[i][j]
                index_1 = i
                index_2 = j
    
    return (index_1, index_2)

In [ ]:
pair_group(create_diff_dict(animals, nodes))

In [ ]:
def calculate_weight(tup):
    '''Determine number of items in a nested tuple'''
    if(type(tup) != tuple):
        return 1
    
    num_values = 0
    
    for i in tup:
        if type(i) == tuple:
            num_values += calculate_weight(i)
        else:
            num_values += 1
            
    return num_values

In [ ]:
def get_distance_to_farthest_node(root):
    if(root.right == None):
        return 0
    
    return root.right[1] + get_distance_to_farthest_node(root.right[0])

In [ ]:
def generate_phylogenetic_tree(diff_dict):
    '''Creates a phylogentic tree from a list of sequences'''
    
    # Create the dictionary of nodes for the phylogentic tree creation
    node_dict = {}

    for species in diff_dict:
        node = Node()
        node.val = species
        node_dict[species] = node

    # Apply UPGMA until all nodes have been combined
    while(len(diff_dict.keys()) > 1):
        # Get most related nodes
        (animal_1, animal_2) = pair_group(diff_dict)
        
        # Add a new row for the new node
        new_node = (animal_1, animal_2)
        diff_dict[new_node] = {}
        
        distance = diff_dict[animal_1][animal_2]
        
        node = Node()
        node.val = new_node
        node.left = (node_dict[animal_1], distance / 2 - get_distance_to_farthest_node(node_dict[animal_1]))
        node.right = (node_dict[animal_2], distance / 2 - get_distance_to_farthest_node(node_dict[animal_2]))
        
        node_dict.pop(animal_1)
        node_dict.pop(animal_2)
        node_dict[new_node] = node
        
        # Delete node pairs from dictionary
        diff_dict[animal_1].pop(animal_2)
        diff_dict[animal_2].pop(animal_1)


        # Calculate the weights for the arithmetic mean
        weight1 = calculate_weight(animal_1)
        weight2 = calculate_weight(animal_2)

        # Populate new node's row
        for i in diff_dict[animal_1].keys():
            diff_dict[new_node][i] = (weight1 * diff_dict[animal_1][i] + weight2 * diff_dict[animal_2][i]) / (weight1 + weight2)

        # Delete old nodes' rows
        diff_dict.pop(animal_1)
        diff_dict.pop(animal_2)

        # Update remaining nodes' distances to the newly added node and delete the remaining
        # references to the old nodes
        for i in diff_dict.keys():
            if(i != new_node):
                diff_dict[i][new_node] = (weight1 * diff_dict[i][animal_1] + weight2 * diff_dict[i][animal_2]) / (weight1 + weight2)
                
                diff_dict[i].pop(animal_1)
                diff_dict[i].pop(animal_2)
    
    # The last created node is the completed tree
    return node

In [ ]:
def create_tree(tree, nodes):
    '''Generates a string based tree from a list of indices given a list of strings
    to map the indices to'''
    
    out = [0, 0]
    for i in range(len(tree)):
        if type(tree[i]) == tuple:
            out[i] = create_tree(tree[i], nodes)
        else:
            out[i] = nodes[tree[i]]
            
    return tuple(out)

In [ ]:
# Create matrix of differences
diff_dict = create_diff_dict(animals, nodes)

# Create Node based tree
index_tree = generate_phylogenetic_tree(diff_dict)
index_tree.val

In [ ]:
def write_tree_xml(root, filename):
    with open(filename + '.xml', 'w') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        f.write('<phyloxml xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.phyloxml.org http://www.phyloxml.org/1.10/phyloxml.xsd" xmlns="http://www.phyloxml.org">\n')
        f.write('<phylogeny rooted="true">\n')
        f.write('<clade>')
        
        write_tree_recursive(root, f)
        
        f.write('</clade>\n')
        f.write('</phylogeny>\n')
        f.write('</phyloxml>\n')

In [ ]:
def write_tree_recursive(root, f):
    if(root.right == None):
        f.write('<name>' + root.val + '</name>\n')
        return
    
    f.write('<clade branch_length="' + str(root.right[1]) + '">\n')
    write_tree_recursive(root.right[0], f)
    f.write('</clade>\n')
    
    f.write('<clade branch_length="' + str(root.left[1]) + '">\n')
    write_tree_recursive(root.left[0], f)
    f.write('</clade>\n')

In [ ]:
# Generate PhyloXML file
write_tree_xml(index_tree, 'test_tree')

# Load and visualize phylogenetic tree
tree = Phylo.read('test_tree.xml', 'phyloxml')
tree.ladderize()
Phylo.draw(tree)

# Part 3 - Creating a Tree of Rodents

In [ ]:
def create_diff_codon_dict(df):
    diff_dict = {}
    
    for species1 in df['SpeciesName'].unique():
        for species2 in df['SpeciesName'].unique():
    
            if(species1 == species2):
                continue

            diff = species_diff(species1, species2)

            if(not species1 in diff_dict):
                diff_dict[species1] = {}

            if(not species2 in diff_dict):
                diff_dict[species2] = {}

            diff_dict[species1][species2] = diff
            diff_dict[species2][species1] = diff
        
    return diff_dict

In [ ]:
create_diff_codon_dict(rodents)

In [ ]:
# Create matrix of differences
diff_dict = create_diff_codon_dict(rodents)

# Create Node based tree
index_tree = generate_phylogenetic_tree(diff_dict)

# Generate PhyloXML file
write_tree_xml(index_tree, 'rodent_tree')

# Load and visualize phylogenetic tree
tree = Phylo.read('rodent_tree.xml', 'phyloxml')
tree.ladderize()
Phylo.draw(tree)

# Results and Discussion

# Next Steps